# 01. Web Scrapping : Cars Raw

The goal of this notebook is to obtain the whole data base for the project by applying web scrapping on the website Autoscout24

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import matplotlib.pyplot as plt

from time import sleep
from time import time
from random import randint

In [2]:
# Links for the different categories
small_cars_url = "https://www.autoscout24.es/lst/?sort=age&desc=1&bcol= 1 &offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page= 1 &cy=E&body=1&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&"
sedan_cars_url = "https://www.autoscout24.es/lst/?sort=age&desc=1&bcol= 1 &offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page= 1 &cy=E&body=6&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&"
familiar_cars_url = "https://www.autoscout24.es/lst/?sort=age&desc=1&bcol= 1 &offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page= 1& cy=E&body=5&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&"
minivan_cars_url = "https://www.autoscout24.es/lst/?sort=age&desc=1&bcol= 1 &offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page= 1 &cy=E&body=12&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&"
coupe_cars_url = "https://www.autoscout24.es/lst/?sort=age&desc=1&bcol= 1 &offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page= 1 &cy=E&body=3&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&"
cabrio_cars_url = "https://www.autoscout24.es/lst/?sort=age&desc=1&bcol= 1 &offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page= 1 &cy=E&body=2&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&"
suv_cars_url = "https://www.autoscout24.es/lst/?sort=age&desc=1&bcol= 1 &offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page= 1 &cy=E&body=4&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&"
van_cars_url = "https://www.autoscout24.es/lst/?sort=age&desc=1&bcol= 1 &offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page= 1 &cy=E&body=13&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&"
other_cars_url = "https://www.autoscout24.es/lst/?sort=age&desc=1&bcol= 1 &offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page= 1 &cy=E&body=7&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&"



In [3]:
seller = np.array(["D","P"])
seller

array(['D', 'P'], dtype='<U1')

In [4]:
seller_dictionary = {'D': 'Dealer', 
                      'P': 'Private' }

The colour is not available for srapping as we cannot see the field in the main search, so the techical solution is to include it on the website link when setting coding the webpage to be scrapped as if we include &bcol=10& on the path we are extracting cars for colour 10 in this case

#example of the colour code in the link:
"https://www.autoscout24.es/lst/?sort=age&desc=1&bcol=1&offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page=1&cy=E&body=12&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&"

In [5]:
colours = np.array([1,2,3,4,5,6,7,10,11,12,13,14,15,16])
colours

array([ 1,  2,  3,  4,  5,  6,  7, 10, 11, 12, 13, 14, 15, 16])

In [6]:
colours_dictionary = {'1': 'beige', 
                      '2': 'blue', 
                      '3': 'brown', 
                      '4': 'bronze', 
                      '5': 'yellow', 
                      '6': 'grey', 
                      '7': 'green',
                      '10': 'red',
                      '11': 'black', 
                      '12': 'silver', 
                      '13': 'burgundy', 
                      '14': 'white', 
                      '15': 'orange', 
                      '16': 'gold'}

In [7]:
body =  np.array([1,6,5,12,3,2,4,13,7])
body

array([ 1,  6,  5, 12,  3,  2,  4, 13,  7])

In [8]:
body_type_dictionary = {'1': 'small', 
                    '6': 'sedan', 
                    '5': 'familiar', 
                    '12': 'minivan', 
                    '3': 'coupe', 
                    '2': 'cabrio', 
                    '4': 'suv',
                    '13': 'van',
                    '7' : 'other'} 

In [9]:
# Analysing urls for the diffenret pages:
# url_pag1 = "https://www.autoscout24.es/lst/?sort=standard&desc=0&ustate=N%2CU&size=20&page=1&cy=E&atype=C&recommended_sorting_based_id=add4f573-8a91-417b-8217-546c0129def6&"
# url_pag2 = "https://www.autoscout24.es/lst/?sort=standard&desc=0&ustate=N%2CU&size=20&page=2&cy=E&atype=C&recommended_sorting_based_id=add4f573-8a91-417b-8217-546c0129def6&"
# url_pag3 = "https://www.autoscout24.es/lst/?sort=standard&desc=0&ustate=N%2CU&size=20&page=3&cy=E&atype=C&recommended_sorting_based_id=add4f573-8a91-417b-8217-546c0129def6&"

# Max page per url = 20 (start: page 1, stop: page 21 -1, step: 1 by 1 as the url changes from page=n to page=n+1 from one page to the following

all_pages = np.arange(1,21,1)

In [10]:
#Defining columns for Dealer cars
title = []
version = []
year = []
kms = []
hp = []
gear = []
fuel = []
fuel_cons = []
co2 =[]
prev_owners = []
price = []
location = []
colour = []
body_type = []
seller_type = []

In [11]:
#Scrapping Dealers

start = time()


for car_colours in colours:
    for car_types in body:
        for all_cars in all_pages:

            all_cars_page = requests.get("https://www.autoscout24.es/lst/?sort=age&desc=1&custtype=D&bcol="+ str(car_colours) + "&offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page=" + str(all_cars) + "&cy=E&body=" + str(car_types) + "&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&")
            all_cars_soup = BeautifulSoup(all_cars_page.text,"lxml")
            all_cars_ads = all_cars_soup.find_all('div', attrs={"class": "cl-list-element cl-list-element-gap"})

            sleep(randint(2,10))

            for element in all_cars_ads:
                try:
                    Title = element.find('h2', attrs={"class":'cldt-summary-makemodel sc-font-bold sc-ellipsis'}).text
                except:
                    Title = ""
                title.append(Title)

                try:
                    Version = element.find('h2', attrs={"class":"cldt-summary-version sc-ellipsis"}).text
                except:
                    Version = ""
                version.append(Version)
                try:
                    Year = element.find('li', attrs={'data-type':'first-registration'}).text
                except:
                    Year = ""
                year.append(Year)
                try:
                    Kms = element.find('li', attrs={"data-type" : "mileage"}).text
                except:
                    Kms = ""
                kms.append(Kms)

                try:
                    Gear = element.find('li', attrs={"data-type":"transmission-type"}).text
                except:
                    Gear = ""
                gear.append(Gear)

                try:
                    Fuel = element.find('li', attrs={"class":"summary_item_no_bottom_line"}).text
                except:
                    Fuel = ""
                fuel.append(Fuel)

                try:
                    Fuel_cons = element.find('li', attrs={"data-type":"combined-consumption"}).text
                except:
                    Fuel_cons = ""
                fuel_cons.append(Fuel_cons)

                try:
                    Co2 = element.find('li', attrs={"data-type":"co2-emission"}).text
                except:
                    Co2 = ""
                co2.append(Co2)

                try:
                    Prev_owners = element.find('li', attrs={"data-type" : "previous-owners"}).text
                except:
                    Prev_owners = ""
                prev_owners.append(Prev_owners)
                try:
                    Price = element.find('span', attrs={"class":"cldt-price sc-font-xl sc-font-bold"}).text
                except:
                    Price = ""
                price.append(Price)

                try:
                    Hp = element.find('li', text =re.compile(r'CV')).text
                except:
                    Hp = ""
                hp.append(Hp)

                try:
                    Location = element.find('span', attrs={"class":"cldf-summary-seller-contact-zip-city"}).text
                except:
                    Location = ""
                location.append(Location)

                try:
                    Car_colours = car_colours

                except :
                    Car_colours = ""
                colour.append(Car_colours)

                try:
                    Car_types = car_types
                except:
                    Car_types = ""
                body_type.append(Car_types)

                        
                        
                        
end = time()

print(end - start)

16549.13506770134


In [12]:
#Creating the DataFrame
dealer_cars_df = pd.DataFrame({"Title":title,
                        "Version":version,
                        "Year": year,
                        "Kms": kms,
                        "Hp": hp,
                        "Fuel_type": fuel,
                        "Gear_type": gear,
                        "Fuel_cons": fuel_cons,
                        "CO2_g/Km": co2,
                        "Previous_owners": prev_owners,
                        "City_ZIP" : location,
                        "Price_EUR": price,
                        "Colour": colour ,
                        "Type" : body_type,
                        "Seller" : "D"})

dealer_cars_df

,Title,Version,Year,Kms,Hp,Fuel_type,Gear_type,Fuel_cons,CO2_g/Km,Previous_owners,City_ZIP,Price_EUR,Colour,Type,Seller
0,Citroen C3,1.4HDI Collection,\n06/2006\n,\n182.510 km\n,\n50 kW (68 CV)\n,\nDiésel\n,\nManual\n,"\n4,4 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,08950 ESPLUGUES DE LLOBREGAT,"\n€ 2.400,-\n",1,1,D
1,Audi A1,1.4 TFSI Attraction 92kW,\n09/2017\n,\n67.478 km\n,\n92 kW (125 CV)\n,\nGasolina\n,\nManual\n,"\n4,9 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,08097 L'Hospitalet de Llobregat,"\n€ 14.990,-\n",1,1,D
2,MINI One,1.5,\n09/2019\n,\n3.211 km\n,\n75 kW (102 CV)\n,\nGasolina\n,\nManual\n,\n- (l/100 km)\n,\n- (g/km)\n,\n- (Propietarios)\n,8130 SANTA PERPETUA DE MOGODA,"\n€ 19.500,-\n",1,1,D
3,Citroen C3,1.5BlueHDi S&S Shine 100,\n10/2020\n,\n3 km\n,\n73 kW (99 CV)\n,\nDiésel\n,\nManual\n,"\n3,7 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,48950 Erandio,"\n€ 16.995,-\n",1,1,D
4,Nissan Micra,1.2 Visia,\n10/2005\n,\n134.970 km\n,\n48 kW (65 CV)\n,\nGasolina\n,\nManual\n,"\n5,9 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,21400 AYAMONTE,"\n€ 2.950,-\n",1,1,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23800,MG MGB,,\n02/1974\n,\n50.000 km\n,\n- kW (- CV)\n,\nGasolina\n,\nManual\n,\n- (l/100 km)\n,\n- (g/km)\n,\n- (Propietarios)\n,8860 CASTELLDEFELS,"\n€ 16.500,-\n",15,7,D
23801,Kia XCeed,1.6 PHEV E-TECH,\n05/2021\n,\n10 km\n,\n104 kW (141 CV)\n,\nOtros\n,\nAutomático\n,\n- (l/100 km)\n,\n- (g/km)\n,\n- (Propietarios)\n,14014 CORDOBA,"\n€ 23.350,-\n",15,7,D
23802,Nissan Micra,DIG-T TEKNA BOSE,\n11/2018\n,\n18.000 km\n,\n66 kW (90 CV)\n,\nGasolina\n,\nManual\n,\n- (l/100 km)\n,\n- (g/km)\n,\n- (Propietarios)\n,14014 CORDOBA,"\n€ 13.500,-\n",15,7,D
23803,Plymouth Road Runner,Superbird,\n01/1970\n,\n77.700 km\n,\n- kW (- CV)\n,\nGasolina\n,\nAutomático\n,\n- (l/100 km)\n,\n- (g/km)\n,\n- (Propietarios)\n,3440 Ibi,"\n€ 220.000,-\n",15,7,D


In [13]:
#Defining columns for Private cars
title = []
version = []
year = []
kms = []
hp = []
gear = []
fuel = []
fuel_cons = []
co2 =[]
prev_owners = []
price = []
location = []
colour = []
body_type = []
seller_type = []



In [14]:
#Scrapping Private

start = time()


for car_colours in colours:
    for car_types in body:
        for all_cars in all_pages:

            all_cars_page = requests.get("https://www.autoscout24.es/lst/?sort=age&desc=1&custtype=P&bcol="+ str(car_colours) + "&offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page=" + str(all_cars) + "&cy=E&body=" + str(car_types) + "&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&")
            all_cars_soup = BeautifulSoup(all_cars_page.text,"lxml")
            all_cars_ads = all_cars_soup.find_all('div', attrs={"class": "cl-list-element cl-list-element-gap"})

            sleep(randint(2,10))

            for element in all_cars_ads:
                try:
                    Title = element.find('h2', attrs={"class":'cldt-summary-makemodel sc-font-bold sc-ellipsis'}).text
                except:
                    Title = ""
                title.append(Title)

                try:
                    Version = element.find('h2', attrs={"class":"cldt-summary-version sc-ellipsis"}).text
                except:
                    Version = ""
                version.append(Version)
                try:
                    Year = element.find('li', attrs={'data-type':'first-registration'}).text
                except:
                    Year = ""
                year.append(Year)
                try:
                    Kms = element.find('li', attrs={"data-type" : "mileage"}).text
                except:
                    Kms = ""
                kms.append(Kms)

                try:
                    Gear = element.find('li', attrs={"data-type":"transmission-type"}).text
                except:
                    Gear = ""
                gear.append(Gear)

                try:
                    Fuel = element.find('li', attrs={"class":"summary_item_no_bottom_line"}).text
                except:
                    Fuel = ""
                fuel.append(Fuel)

                try:
                    Fuel_cons = element.find('li', attrs={"data-type":"combined-consumption"}).text
                except:
                    Fuel_cons = ""
                fuel_cons.append(Fuel_cons)

                try:
                    Co2 = element.find('li', attrs={"data-type":"co2-emission"}).text
                except:
                    Co2 = ""
                co2.append(Co2)

                try:
                    Prev_owners = element.find('li', attrs={"data-type" : "previous-owners"}).text
                except:
                    Prev_owners = ""
                prev_owners.append(Prev_owners)
                try:
                    Price = element.find('span', attrs={"class":"cldt-price sc-font-xl sc-font-bold"}).text
                except:
                    Price = ""
                price.append(Price)

                try:
                    Hp = element.find('li', text =re.compile(r'CV')).text
                except:
                    Hp = ""
                hp.append(Hp)

                try:
                    Location = element.find('span', attrs={"class":"cldt-summary-seller-contact-zip-city"}).text
                except:
                    Location = ""
                location.append(Location)

                try:
                    Car_colours = car_colours

                except :
                    Car_colours = ""
                colour.append(Car_colours)

                try:
                    Car_types = car_types
                except:
                    Car_types = ""
                body_type.append(Car_types)

                        
                        
                        
end = time()

print(end - start)

16235.993126869202


In [15]:
#Creating the DataFrame
private_cars_df = pd.DataFrame({"Title":title,
                        "Version":version,
                        "Year": year,
                        "Kms": kms,
                        "Hp": hp,
                        "Fuel_type": fuel,
                        "Gear_type": gear,
                        "Fuel_cons": fuel_cons,
                        "CO2_g/Km": co2,
                        "Previous_owners": prev_owners,
                        "City_ZIP" : location,
                        "Price_EUR": price,
                        "Colour": colour ,
                        "Type" : body_type,
                        "Seller" : "P"})
private_cars_df

,Title,Version,Year,Kms,Hp,Fuel_type,Gear_type,Fuel_cons,CO2_g/Km,Previous_owners,City_ZIP,Price_EUR,Colour,Type,Seller
0,Fiat 500,1.4 16V 100CV spot,\n07/2010\n,\n89.000 km\n,\n74 kW (101 CV)\n,\nGasolina\n,\nManual\n,\n- (l/100 km)\n,\n- (g/km)\n,\n1 Propietario\n,18001 GRANADA,"\n€ 8.900,-\n",1,1,P
1,Volkswagen Käfer,Oval 1954,\n11/1954\n,\n1.300 km\n,\n26 kW (35 CV)\n,\nGasolina\n,\n- (Cambio)\n,\n- (l/100 km)\n,\n- (g/km)\n,\n- (Propietarios)\n,08860 castelldefels,"\n€ 23.000,-\n",1,1,P
2,Citroen 2CV,2cv 12v,\n01/1977\n,\n10.000 km\n,\n443 kW (602 CV)\n,\nGasolina\n,\nManual\n,\n- (l/100 km)\n,\n- (g/km)\n,\n2 Propietarios\n,08360 Canet de Mar,"\n€ 8.000,-\n",1,1,P
3,Aixam City,Minauto GT,\n10/2016\n,\n36.000 km\n,\n8 kW (11 CV)\n,\nDiésel\n,\nAutomático\n,\n3 l/100 km (mixto)\nPuede obtenerse informac...,\n- (g/km)\n,\n- (Propietarios)\n,28224 Madrid,"\n€ 6.000,-\n",1,1,P
4,Citroen 2CV,,\n10/1977\n,\n60.000 km\n,\n24 kW (33 CV)\n,\nGasolina\n,\n- (Cambio)\n,\n- (l/100 km)\n,\n- (g/km)\n,\n- (Propietarios)\n,08370 Calella,"\n€ 6.000,-\n",1,1,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10543,Ford Transit Connect,FT 230L TDCi 110,\n09/2007\n,\n266.600 km\n,\n81 kW (110 CV)\n,\nDiésel\n,\nManual\n,"\n6,3 l/100 km (mixto)\nPuede obtenerse inform...",\n172 g/km (mixto)\nPuede obtenerse informació...,\n- (Propietarios)\n,24006 León,"\n€ 3.400,-\n",16,13,P
10544,Aixam Mega,,\n01/2004\n,\n32.000 km\n,\n4 kW (5 CV)\n,\nDiésel\n,\n- (Cambio)\n,\n- (l/100 km)\n,\n- (g/km)\n,\n- (Propietarios)\n,08917 Badalona,"\n€ 4.000,-\n",16,13,P
10545,Renault R 5,,\n09/1981\n,\n2.200 km\n,\n950 kW (1.292 CV)\n,\nGasolina\n,\n- (Cambio)\n,\n- (l/100 km)\n,\n- (g/km)\n,\n- (Propietarios)\n,03201 Elche,"\n€ 500,-\n",16,7,P
10546,Mercedes-Benz SL 450,,\n08/1973\n,\n214.000 km\n,\n165 kW (224 CV)\n,\nGasolina\n,\n- (Cambio)\n,\n- (l/100 km)\n,\n- (g/km)\n,\n- (Propietarios)\n,43001 Tarragona,"\n€ 24.500,-\n",16,7,P


In [17]:
#Mergin DataFrames 
all_cars_df = pd.concat([dealer_cars_df,
                         private_cars_df], axis = 0)
#Reset_index
all_cars_df = all_cars_df.reset_index()

In [20]:


all_cars_df

,index,Title,Version,Year,Kms,Hp,Fuel_type,Gear_type,Fuel_cons,CO2_g/Km,Previous_owners,City_ZIP,Price_EUR,Colour,Type,Seller
0,0,Citroen C3,1.4HDI Collection,\n06/2006\n,\n182.510 km\n,\n50 kW (68 CV)\n,\nDiésel\n,\nManual\n,"\n4,4 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,08950 ESPLUGUES DE LLOBREGAT,"\n€ 2.400,-\n",1,1,D
1,1,Audi A1,1.4 TFSI Attraction 92kW,\n09/2017\n,\n67.478 km\n,\n92 kW (125 CV)\n,\nGasolina\n,\nManual\n,"\n4,9 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,08097 L'Hospitalet de Llobregat,"\n€ 14.990,-\n",1,1,D
2,2,MINI One,1.5,\n09/2019\n,\n3.211 km\n,\n75 kW (102 CV)\n,\nGasolina\n,\nManual\n,\n- (l/100 km)\n,\n- (g/km)\n,\n- (Propietarios)\n,8130 SANTA PERPETUA DE MOGODA,"\n€ 19.500,-\n",1,1,D
3,3,Citroen C3,1.5BlueHDi S&S Shine 100,\n10/2020\n,\n3 km\n,\n73 kW (99 CV)\n,\nDiésel\n,\nManual\n,"\n3,7 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,48950 Erandio,"\n€ 16.995,-\n",1,1,D
4,4,Nissan Micra,1.2 Visia,\n10/2005\n,\n134.970 km\n,\n48 kW (65 CV)\n,\nGasolina\n,\nManual\n,"\n5,9 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,21400 AYAMONTE,"\n€ 2.950,-\n",1,1,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34348,10543,Ford Transit Connect,FT 230L TDCi 110,\n09/2007\n,\n266.600 km\n,\n81 kW (110 CV)\n,\nDiésel\n,\nManual\n,"\n6,3 l/100 km (mixto)\nPuede obtenerse inform...",\n172 g/km (mixto)\nPuede obtenerse informació...,\n- (Propietarios)\n,24006 León,"\n€ 3.400,-\n",16,13,P
34349,10544,Aixam Mega,,\n01/2004\n,\n32.000 km\n,\n4 kW (5 CV)\n,\nDiésel\n,\n- (Cambio)\n,\n- (l/100 km)\n,\n- (g/km)\n,\n- (Propietarios)\n,08917 Badalona,"\n€ 4.000,-\n",16,13,P
34350,10545,Renault R 5,,\n09/1981\n,\n2.200 km\n,\n950 kW (1.292 CV)\n,\nGasolina\n,\n- (Cambio)\n,\n- (l/100 km)\n,\n- (g/km)\n,\n- (Propietarios)\n,03201 Elche,"\n€ 500,-\n",16,7,P
34351,10546,Mercedes-Benz SL 450,,\n08/1973\n,\n214.000 km\n,\n165 kW (224 CV)\n,\nGasolina\n,\n- (Cambio)\n,\n- (l/100 km)\n,\n- (g/km)\n,\n- (Propietarios)\n,43001 Tarragona,"\n€ 24.500,-\n",16,7,P


In [21]:
all_cars_df.drop(["index"],axis = 1, inplace = True)

In [22]:
all_cars_df

,Title,Version,Year,Kms,Hp,Fuel_type,Gear_type,Fuel_cons,CO2_g/Km,Previous_owners,City_ZIP,Price_EUR,Colour,Type,Seller
0,Citroen C3,1.4HDI Collection,\n06/2006\n,\n182.510 km\n,\n50 kW (68 CV)\n,\nDiésel\n,\nManual\n,"\n4,4 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,08950 ESPLUGUES DE LLOBREGAT,"\n€ 2.400,-\n",1,1,D
1,Audi A1,1.4 TFSI Attraction 92kW,\n09/2017\n,\n67.478 km\n,\n92 kW (125 CV)\n,\nGasolina\n,\nManual\n,"\n4,9 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,08097 L'Hospitalet de Llobregat,"\n€ 14.990,-\n",1,1,D
2,MINI One,1.5,\n09/2019\n,\n3.211 km\n,\n75 kW (102 CV)\n,\nGasolina\n,\nManual\n,\n- (l/100 km)\n,\n- (g/km)\n,\n- (Propietarios)\n,8130 SANTA PERPETUA DE MOGODA,"\n€ 19.500,-\n",1,1,D
3,Citroen C3,1.5BlueHDi S&S Shine 100,\n10/2020\n,\n3 km\n,\n73 kW (99 CV)\n,\nDiésel\n,\nManual\n,"\n3,7 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,48950 Erandio,"\n€ 16.995,-\n",1,1,D
4,Nissan Micra,1.2 Visia,\n10/2005\n,\n134.970 km\n,\n48 kW (65 CV)\n,\nGasolina\n,\nManual\n,"\n5,9 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,21400 AYAMONTE,"\n€ 2.950,-\n",1,1,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34348,Ford Transit Connect,FT 230L TDCi 110,\n09/2007\n,\n266.600 km\n,\n81 kW (110 CV)\n,\nDiésel\n,\nManual\n,"\n6,3 l/100 km (mixto)\nPuede obtenerse inform...",\n172 g/km (mixto)\nPuede obtenerse informació...,\n- (Propietarios)\n,24006 León,"\n€ 3.400,-\n",16,13,P
34349,Aixam Mega,,\n01/2004\n,\n32.000 km\n,\n4 kW (5 CV)\n,\nDiésel\n,\n- (Cambio)\n,\n- (l/100 km)\n,\n- (g/km)\n,\n- (Propietarios)\n,08917 Badalona,"\n€ 4.000,-\n",16,13,P
34350,Renault R 5,,\n09/1981\n,\n2.200 km\n,\n950 kW (1.292 CV)\n,\nGasolina\n,\n- (Cambio)\n,\n- (l/100 km)\n,\n- (g/km)\n,\n- (Propietarios)\n,03201 Elche,"\n€ 500,-\n",16,7,P
34351,Mercedes-Benz SL 450,,\n08/1973\n,\n214.000 km\n,\n165 kW (224 CV)\n,\nGasolina\n,\n- (Cambio)\n,\n- (l/100 km)\n,\n- (g/km)\n,\n- (Propietarios)\n,43001 Tarragona,"\n€ 24.500,-\n",16,7,P


In [24]:
all_cars_df.to_csv('/users/alfon/Desktop/Master/TFM/Csv/01.cars_raw_def.csv', header=True, index=False)